In [1]:
from bertviz import head_view
from transformers import BertTokenizer, BertModel
import os
import glob
from model import Net 
import torch
import numpy as np
import random

In [2]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [3]:
def show_head_view(model, tokenizer, sentence_a, sentence_b=None):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    if sentence_b:
        token_type_ids = inputs['token_type_ids']
        attention = model(input_ids, token_type_ids=token_type_ids)[-1]
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        attention = model(input_ids)[-1]
        sentence_b_start = None
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
    head_view(attention, tokens, sentence_b_start)

In [4]:
def loadState(ckpt_dir, model=None):
    print("Load state at " + ckpt_dir)
    with open(ckpt_dir, 'rb') as f:
        state = torch.load(f)
        fold_iter_idx = state['fold_iter_idx']
        torch.set_rng_state(state['torch_rng_state'])
        torch.cuda.set_rng_state(state['cuda_rng_state']),
        torch.cuda.set_rng_state_all(state['cuda_rng_state_all']),
        np.random.set_state(state['np_rng_state'])
        random.setstate(state['random_rng_state'])
        if state.get('model') is not None and model is not None:
            model.load_state_dict(state['model'])
pretrained_dir='../BioBERT-Models'
model_whole = Net(
    pretrained_dir= pretrained_dir,
    tag_num=4,
    device='cpu',
)

In [5]:
binPath = glob.glob('checkpoints' + '/*SaveModel*')
binPath

['checkpoints/Joint-RC-NER-SaveModel-Test-fineTune-GNP_fold_0_epoch_8.bin',
 'checkpoints/Joint-RC-NER-SaveModel-Test-fineTune-GNP_fold_0_epoch_6.bin',
 'checkpoints/Joint-RC-NER-SaveModel-Test-fineTune-GNP_fold_0_epoch_9.bin']

In [6]:
binPath[1]

'checkpoints/Joint-RC-NER-SaveModel-Test-fineTune-GNP_fold_0_epoch_6.bin'

In [7]:
loadState(binPath[1], model_whole)

Load state at checkpoints/Joint-RC-NER-SaveModel-Test-fineTune-GNP_fold_0_epoch_6.bin


In [8]:
tmp_dir  = 'tmp_bert'
bert = model_whole.bert
bert.save_pretrained(tmp_dir)

In [12]:
text = ' A GST pull - down assay showed that the direct interaction between Gene_A and Gene_N or Gene_B was lost owing to the mutation .'

In [13]:
do_lower_case = False
model = BertModel.from_pretrained(tmp_dir, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(tmp_dir, do_lower_case=do_lower_case)
show_head_view(model, tokenizer, text)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>